### Importing Libraries

In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Transforming Wikitable into Pandas Dataframe and Cleaning Up

In [21]:
# Copying and Pasting Wikitable
html_string = """"
<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
</td></tr>
<tr>
<td>M7A</td>
<td><a href="/wiki/Queen%27s_Park_(Toronto)" title="Queen&#39;s Park (Toronto)">Queen's Park</a></td>
<td>Not assigned
</td></tr>
<tr>
<td>M8A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Islington_Avenue" class="mw-redirect" title="Islington Avenue">Islington Avenue</a>
</td></tr>
<tr>
<td>M1B</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Rouge,_Toronto" title="Rouge, Toronto">Rouge</a>
</td></tr>
<tr>
<td>M1B</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Malvern,_Toronto" title="Malvern, Toronto">Malvern</a>
</td></tr>
<tr>
<td>M2B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Don Mills North
</td></tr>
<tr>
<td>M4B</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Woodbine_Gardens" class="mw-redirect" title="Woodbine Gardens">Woodbine Gardens</a>
</td></tr>
<tr>
<td>M4B</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Parkview_Hill" class="mw-redirect" title="Parkview Hill">Parkview Hill</a>
</td></tr>
<tr>
<td>M5B</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Ryerson" title="Ryerson">Ryerson</a>
</td></tr>
<tr>
<td>M5B</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Garden District
</td></tr>
<tr>
<td>M6B</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Glencairn,_Ontario" class="mw-redirect" title="Glencairn, Ontario">Glencairn</a>
</td></tr>
<tr>
<td>M7B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8B</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Cloverdale
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Islington" title="Islington">Islington</a>
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Martin Grove
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Princess_Gardens" title="Princess Gardens">Princess Gardens</a>
</td></tr>
<tr>
<td>M9B</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/West_Deane_Park" class="mw-redirect" title="West Deane Park">West Deane Park</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Highland_Creek_(Toronto)" title="Highland Creek (Toronto)">Highland Creek</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Rouge_Hill" class="mw-redirect" title="Rouge Hill">Rouge Hill</a>
</td></tr>
<tr>
<td>M1C</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Port_Union,_Toronto" title="Port Union, Toronto">Port Union</a>
</td></tr>
<tr>
<td>M2C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3C</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Flemingdon_Park" title="Flemingdon Park">Flemingdon Park</a>
</td></tr>
<tr>
<td>M3C</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Don Mills South
</td></tr>
<tr>
<td>M4C</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Woodbine_Heights" class="mw-redirect" title="Woodbine Heights">Woodbine Heights</a>
</td></tr>
<tr>
<td>M5C</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/St._James_Town" title="St. James Town">St. James Town</a>
</td></tr>
<tr>
<td>M6C</td>
<td>York</td>
<td><a href="/wiki/Humewood-Cedarvale" class="mw-redirect" title="Humewood-Cedarvale">Humewood-Cedarvale</a>
</td></tr>
<tr>
<td>M7C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8C</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Bloordale Gardens
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Eringate
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Markland_Wood" title="Markland Wood">Markland Wood</a>
</td></tr>
<tr>
<td>M9C</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Old Burnhamthorpe
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Guildwood
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Morningside,_Toronto" title="Morningside, Toronto">Morningside</a>
</td></tr>
<tr>
<td>M1E</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/West_Hill,_Toronto" title="West Hill, Toronto">West Hill</a>
</td></tr>
<tr>
<td>M2E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4E</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/The_Beaches" title="The Beaches">The Beaches</a>
</td></tr>
<tr>
<td>M5E</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Berczy_Park" title="Berczy Park">Berczy Park</a>
</td></tr>
<tr>
<td>M6E</td>
<td>York</td>
<td>Caledonia-Fairbanks
</td></tr>
<tr>
<td>M7E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9E</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1G</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Woburn,_Toronto" title="Woburn, Toronto">Woburn</a>
</td></tr>
<tr>
<td>M2G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4G</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Leaside" title="Leaside">Leaside</a>
</td></tr>
<tr>
<td>M5G</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Central Bay Street
</td></tr>
<tr>
<td>M6G</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Christie
</td></tr>
<tr>
<td>M7G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9G</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1H</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Woburn,_Toronto" title="Woburn, Toronto">Cedarbrae</a>
</td></tr>
<tr>
<td>M2H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Hillcrest_Village" title="Hillcrest Village">Hillcrest Village</a>
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bathurst_Manor" title="Bathurst Manor">Bathurst Manor</a>
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview North
</td></tr>
<tr>
<td>M3H</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Wilson_Heights,_Toronto" class="mw-redirect" title="Wilson Heights, Toronto">Wilson Heights</a>
</td></tr>
<tr>
<td>M4H</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/Thorncliffe_Park" title="Thorncliffe Park">Thorncliffe Park</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Adelaide" title="Adelaide">Adelaide</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/King" title="King">King</a>
</td></tr>
<tr>
<td>M5H</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Richmond
</td></tr>
<tr>
<td>M6H</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Dovercourt_Village" class="mw-redirect" title="Dovercourt Village">Dovercourt Village</a>
</td></tr>
<tr>
<td>M6H</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>Dufferin
</td></tr>
<tr>
<td>M7H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9H</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1J</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Scarborough_Village" title="Scarborough Village">Scarborough Village</a>
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Fairview
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Henry_Farm" title="Henry Farm">Henry Farm</a>
</td></tr>
<tr>
<td>M2J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Oriole
</td></tr>
<tr>
<td>M3J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Northwood_Park" class="mw-redirect" title="Northwood Park">Northwood Park</a>
</td></tr>
<tr>
<td>M3J</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/York_University" title="York University">York University</a>
</td></tr>
<tr>
<td>M4J</td>
<td><a href="/wiki/East_York" title="East York">East York</a></td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a>
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbourfront East
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Toronto_Islands" title="Toronto Islands">Toronto Islands</a>
</td></tr>
<tr>
<td>M5J</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Union_Station_(Toronto)" title="Union Station (Toronto)">Union Station</a>
</td></tr>
<tr>
<td>M6J</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Little_Portugal,_Toronto" title="Little Portugal, Toronto">Little Portugal</a>
</td></tr>
<tr>
<td>M6J</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Trinity%E2%80%93Bellwoods" title="Trinity–Bellwoods">Trinity</a>
</td></tr>
<tr>
<td>M7J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9J</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>East Birchmount Park
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Ionview" title="Ionview">Ionview</a>
</td></tr>
<tr>
<td>M1K</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Kennedy_Park,_Toronto" class="mw-redirect" title="Kennedy Park, Toronto">Kennedy Park</a>
</td></tr>
<tr>
<td>M2K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bayview_Village" title="Bayview Village">Bayview Village</a>
</td></tr>
<tr>
<td>M3K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/CFB_Toronto" title="CFB Toronto">CFB Toronto</a>
</td></tr>
<tr>
<td>M3K</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview East
</td></tr>
<tr>
<td>M4K</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>The Danforth West
</td></tr>
<tr>
<td>M4K</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/Riverdale,_Toronto" title="Riverdale, Toronto">Riverdale</a>
</td></tr>
<tr>
<td>M5K</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Design_Exchange" title="Design Exchange">Design Exchange</a>
</td></tr>
<tr>
<td>M5K</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Toronto_Dominion_Centre" class="mw-redirect" title="Toronto Dominion Centre">Toronto Dominion Centre</a>
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>Brockton
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Exhibition_Place" title="Exhibition Place">Exhibition Place</a>
</td></tr>
<tr>
<td>M6K</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Parkdale_Village" class="mw-redirect" title="Parkdale Village">Parkdale Village</a>
</td></tr>
<tr>
<td>M7K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9K</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Clairlea" title="Clairlea">Clairlea</a>
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Golden_Mile,_Toronto" title="Golden Mile, Toronto">Golden Mile</a>
</td></tr>
<tr>
<td>M1L</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Oakridge,_Toronto" title="Oakridge, Toronto">Oakridge</a>
</td></tr>
<tr>
<td>M2L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Silver_Hills" class="mw-redirect" title="Silver Hills">Silver Hills</a>
</td></tr>
<tr>
<td>M2L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/York_Mills" title="York Mills">York Mills</a>
</td></tr>
<tr>
<td>M3L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Downsview" title="Downsview">Downsview West</a>
</td></tr>
<tr>
<td>M4L</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>The Beaches West
</td></tr>
<tr>
<td>M4L</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td><a href="/wiki/India_Bazaar" class="mw-redirect" title="India Bazaar">India Bazaar</a>
</td></tr>
<tr>
<td>M5L</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Commerce_Court" title="Commerce Court">Commerce Court</a>
</td></tr>
<tr>
<td>M5L</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Victoria Hotel
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Maple_Leaf_Park" class="mw-redirect" title="Maple Leaf Park">Maple Leaf Park</a>
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>North Park
</td></tr>
<tr>
<td>M6L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Upwood Park
</td></tr>
<tr>
<td>M7L</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8L</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9L</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Humber_Summit" title="Humber Summit">Humber Summit</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Cliffcrest" title="Cliffcrest">Cliffcrest</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Cliffside,_Toronto" title="Cliffside, Toronto">Cliffside</a>
</td></tr>
<tr>
<td>M1M</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Scarborough Village West
</td></tr>
<tr>
<td>M2M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Newtonbrook" title="Newtonbrook">Newtonbrook</a>
</td></tr>
<tr>
<td>M2M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Willowdale,_Toronto" title="Willowdale, Toronto">Willowdale</a>
</td></tr>
<tr>
<td>M3M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview Central
</td></tr>
<tr>
<td>M4M</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>Studio District
</td></tr>
<tr>
<td>M5M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Bedford_Park,_Toronto" title="Bedford Park, Toronto">Bedford Park</a>
</td></tr>
<tr>
<td>M5M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Lawrence Manor East
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>Del Ray
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Keelesdale" class="mw-redirect" title="Keelesdale">Keelesdale</a>
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Mount_Dennis" title="Mount Dennis">Mount Dennis</a>
</td></tr>
<tr>
<td>M6M</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Silverthorn,_Toronto" title="Silverthorn, Toronto">Silverthorn</a>
</td></tr>
<tr>
<td>M7M</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8M</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Emery,_Toronto" class="mw-redirect" title="Emery, Toronto">Emery</a>
</td></tr>
<tr>
<td>M9M</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Humberlea" class="mw-redirect" title="Humberlea">Humberlea</a>
</td></tr>
<tr>
<td>M1N</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Birch_Cliff" title="Birch Cliff">Birch Cliff</a>
</td></tr>
<tr>
<td>M1N</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Cliffside West
</td></tr>
<tr>
<td>M2N</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Willowdale South
</td></tr>
<tr>
<td>M3N</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>Downsview Northwest
</td></tr>
<tr>
<td>M4N</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Lawrence_Park,_Toronto" title="Lawrence Park, Toronto">Lawrence Park</a>
</td></tr>
<tr>
<td>M5N</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Roselawn
</td></tr>
<tr>
<td>M6N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>The Junction North
</td></tr>
<tr>
<td>M6N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td>Runnymede
</td></tr>
<tr>
<td>M7N</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8N</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9N</td>
<td><a href="/wiki/York" title="York">York</a></td>
<td><a href="/wiki/Weston,_Toronto" title="Weston, Toronto">Weston</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Dorset_Park" title="Dorset Park">Dorset Park</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Scarborough_Town_Centre" title="Scarborough Town Centre">Scarborough Town Centre</a>
</td></tr>
<tr>
<td>M1P</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Wexford_Heights" class="mw-redirect" title="Wexford Heights">Wexford Heights</a>
</td></tr>
<tr>
<td>M2P</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>York Mills West
</td></tr>
<tr>
<td>M3P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Davisville North
</td></tr>
<tr>
<td>M5P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Forest_Hill_North" class="mw-redirect" title="Forest Hill North">Forest Hill North</a>
</td></tr>
<tr>
<td>M5P</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Forest Hill West
</td></tr>
<tr>
<td>M6P</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/High_Park" title="High Park">High Park</a>
</td></tr>
<tr>
<td>M6P</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td>The Junction South
</td></tr>
<tr>
<td>M7P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8P</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9P</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Westmount" class="mw-redirect" title="Westmount">Westmount</a>
</td></tr>
<tr>
<td>M1R</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Maryvale,_Toronto" title="Maryvale, Toronto">Maryvale</a>
</td></tr>
<tr>
<td>M1R</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Wexford" title="Wexford">Wexford</a>
</td></tr>
<tr>
<td>M2R</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Willowdale_West" class="mw-redirect" title="Willowdale West">Willowdale West</a>
</td></tr>
<tr>
<td>M3R</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>North Toronto West
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/The_Annex" title="The Annex">The Annex</a>
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>North Midtown
</td></tr>
<tr>
<td>M5R</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Yorkville,_Toronto" title="Yorkville, Toronto">Yorkville</a>
</td></tr>
<tr>
<td>M6R</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Parkdale,_Toronto" title="Parkdale, Toronto">Parkdale</a>
</td></tr>
<tr>
<td>M6R</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Roncesvalles" title="Roncesvalles">Roncesvalles</a>
</td></tr>
<tr>
<td>M7R</td>
<td>Mississauga</td>
<td>Canada Post Gateway Processing Centre
</td></tr>
<tr>
<td>M8R</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Kingsview_Village" title="Kingsview Village">Kingsview Village</a>
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Martin Grove Gardens
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Richview Gardens
</td></tr>
<tr>
<td>M9R</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>St. Phillips
</td></tr>
<tr>
<td>M1S</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a>
</td></tr>
<tr>
<td>M2S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4S</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Davisville
</td></tr>
<tr>
<td>M5S</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbord
</td></tr>
<tr>
<td>M5S</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/University_of_Toronto" title="University of Toronto">University of Toronto</a>
</td></tr>
<tr>
<td>M6S</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Runnymede" title="Runnymede">Runnymede</a>
</td></tr>
<tr>
<td>M6S</td>
<td><a href="/wiki/West_Toronto" title="West Toronto">West Toronto</a></td>
<td><a href="/wiki/Swansea" title="Swansea">Swansea</a>
</td></tr>
<tr>
<td>M7S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9S</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Clarks Corners
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Sullivan
</td></tr>
<tr>
<td>M1T</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Tam_O%27Shanter_%E2%80%93_Sullivan" title="Tam O&#39;Shanter – Sullivan">Tam O'Shanter</a>
</td></tr>
<tr>
<td>M2T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4T</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Moore_Park,_Toronto" title="Moore Park, Toronto">Moore Park</a>
</td></tr>
<tr>
<td>M4T</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Summerhill East
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Chinatown" title="Chinatown">Chinatown</a>
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Grange_Park_(Toronto)" title="Grange Park (Toronto)">Grange Park</a>
</td></tr>
<tr>
<td>M5T</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Kensington_Market" title="Kensington Market">Kensington Market</a>
</td></tr>
<tr>
<td>M6T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9T</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Agincourt_North" class="mw-redirect" title="Agincourt North">Agincourt North</a>
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>L'Amoreaux East
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Milliken,_Ontario" title="Milliken, Ontario">Milliken</a>
</td></tr>
<tr>
<td>M1V</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>Steeles East
</td></tr>
<tr>
<td>M2V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Deer_Park,_Toronto" title="Deer Park, Toronto">Deer Park</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Forest Hill SE
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/Rathnelly" class="mw-redirect" title="Rathnelly">Rathnelly</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td><a href="/wiki/South_Hill,_Toronto" title="South Hill, Toronto">South Hill</a>
</td></tr>
<tr>
<td>M4V</td>
<td><a href="/wiki/Central_Toronto" class="mw-redirect" title="Central Toronto">Central Toronto</a></td>
<td>Summerhill West
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/CN_Tower" title="CN Tower">CN Tower</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Bathurst Quay
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Island airport
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Harbourfront West
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/King_and_Spadina" class="mw-redirect" title="King and Spadina">King and Spadina</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Railway_Lands" title="Railway Lands">Railway Lands</a>
</td></tr>
<tr>
<td>M5V</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/South_Niagara" class="mw-redirect" title="South Niagara">South Niagara</a>
</td></tr>
<tr>
<td>M6V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7V</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Humber Bay Shores
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Mimico South
</td></tr>
<tr>
<td>M8V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/New_Toronto" title="New Toronto">New Toronto</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Albion Gardens
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Beaumond_Heights" class="mw-redirect" title="Beaumond Heights">Beaumond Heights</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Humbergate
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mount_Olive-Silverstone-Jamestown" class="mw-redirect" title="Mount Olive-Silverstone-Jamestown">Jamestown</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mount_Olive-Silverstone-Jamestown" class="mw-redirect" title="Mount Olive-Silverstone-Jamestown">Mount Olive</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Silverstone" title="Silverstone">Silverstone</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/South_Steeles" class="mw-redirect" title="South Steeles">South Steeles</a>
</td></tr>
<tr>
<td>M9V</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Thistletown" title="Thistletown">Thistletown</a>
</td></tr>
<tr>
<td>M1W</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td>L'Amoreaux West
</td></tr>
<tr>
<td>M1W</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Steeles_West" class="mw-redirect" title="Steeles West">Steeles West</a>
</td></tr>
<tr>
<td>M2W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4W</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Rosedale,_Toronto" title="Rosedale, Toronto">Rosedale</a>
</td></tr>
<tr>
<td>M5W</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td>Stn A PO Boxes 25 The Esplanade
</td></tr>
<tr>
<td>M6W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7W</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Alderwood,_Toronto" title="Alderwood, Toronto">Alderwood</a>
</td></tr>
<tr>
<td>M8W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Long_Branch,_Toronto" title="Long Branch, Toronto">Long Branch</a>
</td></tr>
<tr>
<td>M9W</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Northwest" class="mw-redirect" title="Northwest">Northwest</a>
</td></tr>
<tr>
<td>M1X</td>
<td><a href="/wiki/Scarborough,_Toronto" title="Scarborough, Toronto">Scarborough</a></td>
<td><a href="/wiki/Upper_Rouge" class="mw-redirect" title="Upper Rouge">Upper Rouge</a>
</td></tr>
<tr>
<td>M2X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Cabbagetown,_Toronto" title="Cabbagetown, Toronto">Cabbagetown</a>
</td></tr>
<tr>
<td>M4X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/St._James_Town" title="St. James Town">St. James Town</a>
</td></tr>
<tr>
<td>M5X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/First_Canadian_Place" title="First Canadian Place">First Canadian Place</a>
</td></tr>
<tr>
<td>M5X</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Underground_city" title="Underground city">Underground city</a>
</td></tr>
<tr>
<td>M6X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Kingsway" title="The Kingsway">The Kingsway</a>
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Montgomery Road
</td></tr>
<tr>
<td>M8X</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Old Mill North
</td></tr>
<tr>
<td>M9X</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4Y</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Church_and_Wellesley" title="Church and Wellesley">Church and Wellesley</a>
</td></tr>
<tr>
<td>M5Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M6Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7Y</td>
<td><a href="/wiki/East_Toronto" title="East Toronto">East Toronto</a></td>
<td>Business Reply Mail Processing Centre 969 Eastern
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Humber_Bay" title="Humber Bay">Humber Bay</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Kingsmills_Park" title="Kingsmills Park">King's Mill Park</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Kingsway Park South East
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mimico" title="Mimico">Mimico NE</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Old_Mill,_Toronto" title="Old Mill, Toronto">Old Mill South</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway East</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Fairmont_Royal_York_Hotel" class="mw-redirect" title="Fairmont Royal York Hotel">Royal York South East</a>
</td></tr>
<tr>
<td>M8Y</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Sunnylea" class="mw-redirect" title="Sunnylea">Sunnylea</a>
</td></tr>
<tr>
<td>M9Y</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M1Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M4Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M5Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M6Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M7Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Kingsway Park South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Mimico" title="Mimico">Mimico NW</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway West</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Bloor" title="Bloor">South of Bloor</a>
</td></tr>
<tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
</tbody></table>
"""

In [67]:
# Transforming Wikitable into Pandas Dataframe
from IPython.display import display_html
display_html(html_string, raw=True)

neighbourhoods = pd.read_html(html_string, header = 0)
neighbourhoods = neighbourhoods[0]
neighbourhoods

"
 
 
 Postcode 
 Borough 
 Neighbourhood
 
 
 M1A 
 Not assigned 
 Not assigned
 
 
 M2A 
 Not assigned 
 Not assigned
 
 
 M3A 
 North York 
 Parkwoods 
 
 
 M4A 
 North York 
 Victoria Village 
 
 
 M5A 
 Downtown Toronto 
 Harbourfront 
 
 
 M5A 
 Downtown Toronto 
 Regent Park 
 
 
 M6A 
 North York 
 Lawrence Heights 
 
 
 M6A 
 North York 
 Lawrence Manor 
 
 
 M7A 
 Queen's Park 
 Not assigned
 
 
 M8A 
 Not assigned 
 Not assigned
 
 
 M9A 
 Etobicoke 
 Islington Avenue 
 
 
 M1B 
 Scarborough 
 Rouge 
 
 
 M1B 
 Scarborough 
 Malvern 
 
 
 M2B 
 Not assigned 
 Not assigned
 
 
 M3B 
 North York 
 Don Mills North
 
 
 M4B 
 East York 
 Woodbine Gardens 
 
 
 M4B 
 East York 
 Parkview Hill 
 
 
 M5B 
 Downtown Toronto 
 Ryerson 
 
 
 M5B 
 Downtown Toronto 
 Garden District
 
 
 M6B 
 North York 
 Glencairn 
 
 
 M7B 
 Not assigned 
 Not assigned
 
 
 M8B 
 Not assigned 
 Not assigned
 
 
 M9B 
 Etobicoke 
 Cloverdale
 
 
 M9B 
 Etobicoke 
 Islington 
 
 
 M9B 
 Etobicoke 
 Martin Grove
 
 
 M9B 
 Etobicoke 
 Princess Gardens 
 
 
 M9B 
 Etobicoke 
 West Deane Park 
 
 
 M1C 
 Scarborough 
 Highland Creek 
 
 
 M1C 
 Scarborough 
 Rouge Hill 
 
 
 M1C 
 Scarborough 
 Port Union 
 
 
 M2C 
 Not assigned 
 Not assigned
 
 
 M3C 
 North York 
 Flemingdon Park 
 
 
 M3C 
 North York 
 Don Mills South
 
 
 M4C 
 East York 
 Woodbine Heights 
 
 
 M5C 
 Downtown Toronto 
 St. James Town 
 
 
 M6C 
 York 
 Humewood-Cedarvale 
 
 
 M7C 
 Not assigned 
 Not assigned
 
 
 M8C 
 Not assigned 
 Not assigned
 
 
 M9C 
 Etobicoke 
 Bloordale Gardens
 
 
 M9C 
 Etobicoke 
 Eringate
 
 
 M9C 
 Etobicoke 
 Markland Wood 
 
 
 M9C 
 Etobicoke 
 Old Burnhamthorpe
 
 
 M1E 
 Scarborough 
 Guildwood
 
 
 M1E 
 Scarborough 
 Morningside 
 
 
 M1E 
 Scarborough 
 West Hill 
 
 
 M2E 
 Not assigned 
 Not assigned
 
 
 M3E 
 Not assigned 
 Not assigned
 
 
 M4E 
 East Toronto 
 The Beaches 
 
 
 M5E 
 Downtown Toronto 
 Berczy Park 
 
 
 M6E 
 York 
 Caledonia-Fairbanks
 
 
 M7E 
 Not assigned 
 Not assigned
 
 
 M8E 
 Not assigned 
 Not assigned
 
 
 M9E 
 Not assigned 
 Not assigned
 
 
 M1G 
 Scarborough 
 Woburn 
 
 
 M2G 
 Not assigned 
 Not assigned
 
 
 M3G 
 Not assigned 
 Not assigned
 
 
 M4G 
 East York 
 Leaside 
 
 
 M5G 
 Downtown Toronto 
 Central Bay Street
 
 
 M6G 
 Downtown Toronto 
 Christie
 
 
 M7G 
 Not assigned 
 Not assigned
 
 
 M8G 
 Not assigned 
 Not assigned
 
 
 M9G 
 Not assigned 
 Not assigned
 
 
 M1H 
 Scarborough 
 Cedarbrae 
 
 
 M2H 
 North York 
 Hillcrest Village 
 
 
 M3H 
 North York 
 Bathurst Manor 
 
 
 M3H 
 North York 
 Downsview North
 
 
 M3H 
 North York 
 Wilson Heights 
 
 
 M4H 
 East York 
 Thorncliffe Park 
 
 
 M5H 
 Downtown Toronto 
 Adelaide 
 
 
 M5H 
 Downtown Toronto 
 King 
 
 
 M5H 
 Downtown Toronto 
 Richmond
 
 
 M6H 
 West Toronto 
 Dovercourt Village 
 
 
 M6H 
 West Toronto 
 Dufferin
 
 
 M7H 
 Not assigned 
 Not assigned
 
 
 M8H 
 Not assigned 
 Not assigned
 
 
 M9H 
 Not assigned 
 Not assigned
 
 
 M1J 
 Scarborough 
 Scarborough Village 
 
 
 M2J 
 North York 
 Fairview
 
 
 M2J 
 North York 
 Henry Farm 
 
 
 M2J 
 North York 
 Oriole
 
 
 M3J 
 North York 
 Northwood Park 
 
 
 M3J 
 North York 
 York University 
 
 
 M4J 
 East York 
 East Toronto 
 
 
 M5J 
 Downtown Toronto 
 Harbourfront East
 
 
 M5J 
 Downtown Toronto 
 Toronto Islands 
 
 
 M5J 
 Downtown Toronto 
 Union Station 
 
 
 M6J 
 West Toronto 
 Little Portugal 
 
 
 M6J 
 West Toronto 
 Trinity 
 
 
 M7J 
 Not assigned 
 Not assigned
 
 
 M8J 
 Not assigned 
 Not assigned
 
 
 M9J 
 Not assigned 
 Not assigned
 
 
 M1K 
 Scarborough 
 East Birchmount Park
 
 
 M1K 
 Scarborough 
 Ionview 
 
 
 M1K 
 Scarborough 
 Kennedy Park 
 
 
 M2K 
 North York 
 Bayview Village 
 
 
 M3K 
 North York 
 CFB Toronto 
 
 
 M3K 
 North York 
 Downsview East
 
 
 M4K 
 East Toronto 
 The Danforth West
 
 
 M4K 
 East Toronto 
 Riverdale 
 
 
 M5K 
 Downtown Toronto 
 Design Exchange 
 
 
 M5K 
 Downtown Toronto 
 

Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
9        M8A      Not assigned   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
13       M2B      Not assigned   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
20       M7B      Not assigned   
21       M8B      Not assigned   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C       Scarborough   
29       M1C       Scarborough   
30       M2C      Not assigned   
31       M3C        North York   
32       M3C        North York   
33       M4C         East York   
34       M5C  Downtown Toronto   
35       M6C              York   
36       M7C      Not assigned   
37       M8C      Not assigned   
38       M9C         Etobicoke   
39       M9C         Etobicoke   
40       M9C         Etobicoke   
41       M9C         Etobicoke   
42       M1E       Scarborough   
43       M1E       Scarborough   
44       M1E       Scarborough   
45       M2E      Not assigned   
46       M3E      Not assigned   
47       M4E      East Toronto   
48       M5E  Downtown Toronto   
49       M6E              York   
50       M7E      Not assigned   
51       M8E      Not assigned   
52       M9E      Not assigned   
53       M1G       Scarborough   
54       M2G      Not assigned   
55       M3G      Not assigned   
56       M4G         East York   
57       M5G  Downtown Toronto   
58       M6G  Downtown Toronto   
59       M7G      Not assigned   
60       M8G      Not assigned   
61       M9G      Not assigned   
62       M1H       Scarborough   
63       M2H        North York   
64       M3H        North York   
65       M3H        North York   
66       M3H        North York   
67       M4H         East York   
68       M5H  Downtown Toronto   
69       M5H  Downtown Toronto   
70       M5H  Downtown Toronto   
71       M6H      West Toronto   
72       M6H      West Toronto   
73       M7H      Not assigned   
74       M8H      Not assigned   
75       M9H      Not assigned   
76       M1J       Scarborough   
77       M2J        North York   
78       M2J        North York   
79       M2J        North York   
80       M3J        North York   
81       M3J        North York   
82       M4J         East York   
83       M5J  Downtown Toronto   
84       M5J  Downtown Toronto   
85       M5J  Downtown Toronto   
86       M6J      West Toronto   
87       M6J      West Toronto   
88       M7J      Not assigned   
89       M8J      Not assigned   
90       M9J      Not assigned   
91       M1K       Scarborough   
92       M1K       Scarborough   
93       M1K       Scarborough   
94       M2K        North York   
95       M3K        North York   
96       M3K        North York   
97       M4K      East Toronto   
98       M4K      East Toronto   
99       M5K  Downtown Toronto   
100      M5K  Downtown Toronto   
101      M6K      West Toronto   
102      M6K      West Toronto   
103      M6K      West Toronto   
104      M7K      Not assigned   
105      M8K      Not assigned   
106      M9K      Not assigned   
107      M1L       Scarborough   
108      M1L       Scarborough   
109      M1L       Scarborough   
110      M2L        North York   
111      M2L        North York   
112      M3L        North York   
113      M4L      East Toronto   
114      M4L      East Toronto   
115      M5L  Downtown Toronto   
116      M5L  Downtown Tor

In [68]:
# Dropping 'Not assigned' Boroughs
neighbourhoods = neighbourhoods[neighbourhoods.Borough != 'Not assigned']
neighbourhoods

Postcode           Borough  \
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C       Scarborough   
29       M1C       Scarborough   
31       M3C        North York   
32       M3C        North York   
33       M4C         East York   
34       M5C  Downtown Toronto   
35       M6C              York   
38       M9C         Etobicoke   
39       M9C         Etobicoke   
40       M9C         Etobicoke   
41       M9C         Etobicoke   
42       M1E       Scarborough   
43       M1E       Scarborough   
44       M1E       Scarborough   
47       M4E      East Toronto   
48       M5E  Downtown Toronto   
49       M6E              York   
53       M1G       Scarborough   
56       M4G         East York   
57       M5G  Downtown Toronto   
58       M6G  Downtown Toronto   
62       M1H       Scarborough   
63       M2H        North York   
64       M3H        North York   
65       M3H        North York   
66       M3H        North York   
67       M4H         East York   
68       M5H  Downtown Toronto   
69       M5H  Downtown Toronto   
70       M5H  Downtown Toronto   
71       M6H      West Toronto   
72       M6H      West Toronto   
76       M1J       Scarborough   
77       M2J        North York   
78       M2J        North York   
79       M2J        North York   
80       M3J        North York   
81       M3J        North York   
82       M4J         East York   
83       M5J  Downtown Toronto   
84       M5J  Downtown Toronto   
85       M5J  Downtown Toronto   
86       M6J      West Toronto   
87       M6J      West Toronto   
91       M1K       Scarborough   
92       M1K       Scarborough   
93       M1K       Scarborough   
94       M2K        North York   
95       M3K        North York   
96       M3K        North York   
97       M4K      East Toronto   
98       M4K      East Toronto   
99       M5K  Downtown Toronto   
100      M5K  Downtown Toronto   
101      M6K      West Toronto   
102      M6K      West Toronto   
103      M6K      West Toronto   
107      M1L       Scarborough   
108      M1L       Scarborough   
109      M1L       Scarborough   
110      M2L        North York   
111      M2L        North York   
112      M3L        North York   
113      M4L      East Toronto   
114      M4L      East Toronto   
115      M5L  Downtown Toronto   
116      M5L  Downtown Toronto   
117      M6L        North York   
118      M6L        North York   
119      M6L        North York   
122      M9L        North York   
123      M1M       Scarborough   
124      M1M       Scarborough   
125      M1M       Scarborough   
126      M2M        North York   
127      M2M        North York   
128      M3M        North York   
129      M4M      East Toronto   
130      M5M        North York   
131      M5M        North York   
132      M6M              York   
133      M6M              York   
134      M6M              York   
135      M6M              York   
138      M9M        North York   
139      M9M        North York   
140      M1N       Scarborough   
141      M1N       Scarborough   
142      M2N        North York   
143      M3N        North York   
144      M4N   Central Toronto   
145      M5N   Central Toronto   
146      M6N              York   
147      M6N              York   
150      M9N              

In [69]:
# Grouping based on Borough
neighbourhoods = neighbourhoods.groupby(['Postcode','Borough'], sort=False).Neighbourhood.apply(', '.join).reset_index()
neighbourhoods

Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H        North York   
29       M4H         East York   
30       M5H  Downtown Toronto   
31       M6H      West Toronto   
32       M1J       Scarborough   
33       M2J        North York   
34       M3J        North York   
35       M4J         East York   
36       M5J  Downtown Toronto   
37       M6J      West Toronto   
38       M1K       Scarborough   
39       M2K        North York   
40       M3K        North York   
41       M4K      East Toronto   
42       M5K  Downtown Toronto   
43       M6K      West Toronto   
44       M1L       Scarborough   
45       M2L        North York   
46       M3L        North York   
47       M4L      East Toronto   
48       M5L  Downtown Toronto   
49       M6L        North York   
50       M9L        North York   
51       M1M       Scarborough   
52       M2M        North York   
53       M3M        North York   
54       M4M      East Toronto   
55       M5M        North York   
56       M6M              York   
57       M9M        North York   
58       M1N       Scarborough   
59       M2N        North York   
60       M3N        North York   
61       M4N   Central Toronto   
62       M5N   Central Toronto   
63       M6N              York   
64       M9N              York   
65       M1P       Scarborough   
66       M2P        North York   
67       M4P   Central Toronto   
68       M5P   Central Toronto   
69       M6P      West Toronto   
70       M9P         Etobicoke   
71       M1R       Scarborough   
72       M2R        North York   
73       M4R   Central Toronto   
74       M5R   Central Toronto   
75       M6R      West Toronto   
76       M7R       Mississauga   
77       M9R         Etobicoke   
78       M1S       Scarborough   
79       M4S   Central Toronto   
80       M5S  Downtown Toronto   
81       M6S      West Toronto   
82       M1T       Scarborough   
83       M4T   Central Toronto   
84       M5T  Downtown Toronto   
85       M1V       Scarborough   
86       M4V   Central Toronto   
87       M5V  Downtown Toronto   
88       M8V         Etobicoke   
89       M9V         Etobicoke   
90       M1W       Scarborough   
91       M4W  Downtown Toronto   
92       M5W  Downtown Toronto   
93       M8W         Etobicoke   
94       M9W         Etobicoke   
95       M1X       Scarborough   
96       M4X  Downtown Toronto   
97       M5X  Downtown Toronto   
98       M8X         Etobicoke   
99       M4Y  Downtown Toronto   
100      M7Y      East Toronto   
101      M8Y         Etobicoke   
102      M8Z         Etobicoke   

                                         Neighbourhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Harbourfront, Regent Park  
3                     Lawrence Heights, Lawrence Manor  
4                                         Not assigned  
5                                     Islington Avenue  
6                                       Rouge, Malvern  
7          

In [70]:
# Replacing 'Not assigned' in the 'Neighbourhood' column
rep = neighbourhoods['Borough']
neighbourhoods['Neighbourhood'] = rep.where(neighbourhoods['Neighbourhood'] == 'Not assigned', other = neighbourhoods['Neighbourhood'])
neighbourhoods

Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H        North York   
29       M4H         East York   
30       M5H  Downtown Toronto   
31       M6H      West Toronto   
32       M1J       Scarborough   
33       M2J        North York   
34       M3J        North York   
35       M4J         East York   
36       M5J  Downtown Toronto   
37       M6J      West Toronto   
38       M1K       Scarborough   
39       M2K        North York   
40       M3K        North York   
41       M4K      East Toronto   
42       M5K  Downtown Toronto   
43       M6K      West Toronto   
44       M1L       Scarborough   
45       M2L        North York   
46       M3L        North York   
47       M4L      East Toronto   
48       M5L  Downtown Toronto   
49       M6L        North York   
50       M9L        North York   
51       M1M       Scarborough   
52       M2M        North York   
53       M3M        North York   
54       M4M      East Toronto   
55       M5M        North York   
56       M6M              York   
57       M9M        North York   
58       M1N       Scarborough   
59       M2N        North York   
60       M3N        North York   
61       M4N   Central Toronto   
62       M5N   Central Toronto   
63       M6N              York   
64       M9N              York   
65       M1P       Scarborough   
66       M2P        North York   
67       M4P   Central Toronto   
68       M5P   Central Toronto   
69       M6P      West Toronto   
70       M9P         Etobicoke   
71       M1R       Scarborough   
72       M2R        North York   
73       M4R   Central Toronto   
74       M5R   Central Toronto   
75       M6R      West Toronto   
76       M7R       Mississauga   
77       M9R         Etobicoke   
78       M1S       Scarborough   
79       M4S   Central Toronto   
80       M5S  Downtown Toronto   
81       M6S      West Toronto   
82       M1T       Scarborough   
83       M4T   Central Toronto   
84       M5T  Downtown Toronto   
85       M1V       Scarborough   
86       M4V   Central Toronto   
87       M5V  Downtown Toronto   
88       M8V         Etobicoke   
89       M9V         Etobicoke   
90       M1W       Scarborough   
91       M4W  Downtown Toronto   
92       M5W  Downtown Toronto   
93       M8W         Etobicoke   
94       M9W         Etobicoke   
95       M1X       Scarborough   
96       M4X  Downtown Toronto   
97       M5X  Downtown Toronto   
98       M8X         Etobicoke   
99       M4Y  Downtown Toronto   
100      M7Y      East Toronto   
101      M8Y         Etobicoke   
102      M8Z         Etobicoke   

                                         Neighbourhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Harbourfront, Regent Park  
3                     Lawrence Heights, Lawrence Manor  
4                                         Queen's Park  
5                                     Islington Avenue  
6                                       Rouge, Malvern  
7          

In [72]:
# Dimensions of the Dataframe
neighbourhoods.shape

(103, 3)